In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

In [3]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [4]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [5]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [6]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='4fabf654-1d60-4d08-b790-43ab88ddb9a6'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T18:49:36.701650928Z', 'done': True, 'done_reason': 'stop', 'total_duration': 49828796031, 'load_duration': 14998228599, 'prompt_eval_count': 245, 'prompt_eval_duration': 28389019271, 'eval_count': 22, 'eval_duration': 6340464086, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b99ca-3e14-7533-bebd-24d4bf54e6c8-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'f817f1bd-be7a-4bfc-a74b-1546b25ce8a0', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 245, 'output_tokens': 22, 'total_tokens': 267}),
              ToolMessage(content='The square root of 456.0 is approximately 21.35.', name='call_subagent_1', id='1d345a07-

In [8]:
print(response["messages"][-1].content)

I called a subagent to calculate the square root for you. The result is 21.35.
